# Clean all features with text data

In [65]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
from google.colab import files
from google.colab import drive

#read data from Google Drive
#drive.mount('/content/drive')

#Get ML data from Google Drive
df = pd.read_excel('/content/drive/My Drive/ML/ml_data.xlsx')

#download stopwords
#nltk.download('stopwords')


custom_stopwords = ["i.e", "e.g.", "e.g", "in fact", "=>", "even", "n\'t", "also", "a.k.a", "\'s", "would", "could", "still", 
"see", "can't", "might", "especially", "though",  "customtools", "ct", "sw", "solidworks", "sp"]
stop_words = set(stopwords.words("english")+list(punctuation)+custom_stopwords)

#to lower case, removes hyperlinks, tags, hyphens, line breaks and spaces, numbers, special chars, one letter words and stopwords
def initial_clean_text_data(feature):
    no_hyperlinks = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
    \(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', feature, flags=re.MULTILINE)
    no_tags = re.sub('<[^>]*>', '', no_hyperlinks)
    no_hyphens = no_tags.replace('-', '')
    no_line_breaks_spaces = no_hyphens.replace('\n', ' ').replace('\r', '')
    no_numbers = re.sub('[0-9]', '', no_line_breaks_spaces)
    no_special_chars = re.sub('[^A-Za-z0-9]+', ' ', no_numbers)
    no_one_letter_words = re.sub(r"\b[a-zA-Z]\b", ' ', no_special_chars)
    no_stop_words = ' '.join([word for word in no_one_letter_words.split() if word not in (stop_words)])

    return no_stop_words

#Change all nan to 'u', which means 'unknown' in steps_to_reproduce and additional_information texts so that initial_clean_text_data can be performed
df['steps_to_reproduce'].replace(np.nan, "u", inplace=True)
df['additional_information'].replace(np.nan, "u", inplace=True)

cleaned_sents_description = df['description'].str.lower().apply(initial_clean_text_data)
cleaned_sents_summary = df['summary'].str.lower().apply(initial_clean_text_data)
cleaned_sents_reproduce = df['steps_to_reproduce'].str.lower().apply(initial_clean_text_data)
cleaned_sents_additional_information = df['additional_information'].str.lower().apply(initial_clean_text_data)

#replace all empties with 'u'
cleaned_sents_reproduce = cleaned_sents_reproduce.replace(r'^\s*$', 'u', regex=True)
cleaned_sents_additional_information = cleaned_sents_additional_information.replace(r'^\s*$', 'u', regex=True)
#print(cleaned_sents_reproduce[0])
#print(cleaned_sents_additional_information[0])

#time_estimation = 0 there is no time estimation for SPR
#time_estimation = 1 there is a time_estimate for SPR
cleaned_time_estimation = df['time_estimation'].fillna(0)
cleaned_time_estimation[cleaned_time_estimation != 0] = 1
#time_estimation to type integer
cleaned_time_estimation = cleaned_time_estimation.astype(int)

#tags = 0 there are no tag/tags for SPR
#tags = 1 there are tag/tags for SPR
cleaned_tags = df['tag_id'].fillna(0)
cleaned_tags[cleaned_tags != 0] = 1
#time_estimation to type integer
cleaned_tags = cleaned_tags.astype(int)


#print(cleaned_time_estimation)
#print(cleaned_tags[cleaned_tags == 0])

#100 most common words to verify there are no significant stopwords anymore
print(Counter(" ".join(cleaned_sents_description).split()).most_common(100))
print(Counter(" ".join(cleaned_sents_summary).split()).most_common(100))
#print(Counter(" ".join(cleaned_sents_reproduce).split()).most_common(100))
#print(Counter(" ".join(cleaned_sents_additional_information).split()).most_common(100))
#time_estimation and tags values
#print(Counter(cleaned_time_estimation).most_common(100))
#print(Counter(cleaned_tags).most_common(100))

[('file', 549), ('user', 508), ('database', 351), ('dialog', 333), ('drawing', 330), ('profile', 319), ('attached', 318), ('property', 315), ('value', 314), ('error', 309), ('part', 289), ('properties', 282), ('picture', 271), ('files', 244), ('using', 241), ('selected', 228), ('values', 223), ('conversion', 218), ('box', 214), ('name', 214), ('new', 213), ('assembly', 210), ('message', 206), ('list', 203), ('options', 196), ('sheet', 196), ('customproperties', 195), ('one', 190), ('model', 188), ('pane', 187), ('customer', 181), ('crash', 181), ('project', 179), ('export', 173), ('search', 172), ('open', 171), ('option', 170), ('save', 168), ('folder', 165), ('shown', 163), ('used', 160), ('seems', 160), ('version', 150), ('default', 149), ('document', 146), ('button', 138), ('copy', 134), ('filename', 129), ('currently', 128), ('use', 127), ('information', 122), ('get', 122), ('case', 121), ('configuration', 121), ('work', 120), ('customproperty', 120), ('drawings', 116), ('dxf', 116

# Classify and save cleaned data to Excel
*   0 = (0-2] h
*   1 = (2-5] h
*   2 = (5-8] h
*   4 = (8-15] h
*   5 = (15h+






In [56]:
classified_hours = pd.cut(df['total_hours'], bins=[0, 2, 5, 8, 15, 133], labels=[0, 1, 2, 3, 4])
df['total_hours'] = classified_hours

#save modified text data columns as modified_ml_data.xlsx
df['description'] = cleaned_sents_description
df['summary'] = cleaned_sents_summary
df['steps_to_reproduce'] = cleaned_sents_reproduce
df['additional_information'] = cleaned_sents_additional_information
df['time_estimation'] = cleaned_time_estimation
df['tag_id'] = cleaned_tags

df.to_excel('/content/drive/My Drive/ML/modified_ml_data.xlsx')

#print(df['description'][0])